In [1]:
# Author: Roi Yehoshua <roiyeho@gmail.com>
# Date: February 2024
# License: MIT

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)

Loading and Exploring the Dataset

In [3]:
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(as_frame=True, return_X_y=True)

In [4]:
housing_df = pd.concat([X, y], axis=1)
housing_df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [5]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   MedInc       20640 non-null  float64
 1   HouseAge     20640 non-null  float64
 2   AveRooms     20640 non-null  float64
 3   AveBedrms    20640 non-null  float64
 4   Population   20640 non-null  float64
 5   AveOccup     20640 non-null  float64
 6   Latitude     20640 non-null  float64
 7   Longitude    20640 non-null  float64
 8   MedHouseVal  20640 non-null  float64
dtypes: float64(9)
memory usage: 1.4 MB


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Building the model

In [7]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [8]:
print('Intercept:', np.round(reg.intercept_, 4))
print('Coefficients:', np.round(reg.coef_, 4))

Intercept: -37.0233
Coefficients: [ 0.4487  0.0097 -0.1233  0.7831 -0.     -0.0035 -0.4198 -0.4337]


In [9]:
# Normalize the coefficients
normalized_coef = reg.coef_ / X_train.std(axis=0)
print(np.round(normalized_coef, 4))

MedInc        0.2356
HouseAge      0.0008
AveRooms     -0.0517
AveBedrms     1.8078
Population   -0.0000
AveOccup     -0.0003
Latitude     -0.1965
Longitude    -0.2162
dtype: float64


Evaluating the model

In [10]:
from sklearn.metrics import root_mean_squared_error as RMSE
 
# Evaluation on the training set
train_rmse = RMSE(y_train, reg.predict(X_train))
train_r2 = reg.score(X_train, y_train)
print(f'Train RMSE: {train_rmse:.4f}, R2: {train_r2:.4f}')

# Evaluation on the test set
test_rmse = RMSE(y_test, reg.predict(X_test))
test_r2 = reg.score(X_test, y_test)
print(f'Test RMSE: {test_rmse:.4f}, R2: {test_r2:.4f}')  

Train RMSE: 0.7197, R2: 0.6126
Test RMSE: 0.7456, R2: 0.5758


In [11]:
print('Mean house price in the test set:', y_test.mean())

Mean house price in the test set: 2.0550030959302323


Feature Engineering

In [12]:
X['RoomsPerIndividual'] = X['AveRooms'] / X['AveOccup']

In [13]:
correlations = X.corrwith(y).sort_values(ascending=False)
print(correlations)

MedInc                0.688075
RoomsPerIndividual    0.209482
AveRooms              0.151948
HouseAge              0.105623
AveOccup             -0.023737
Population           -0.024650
Longitude            -0.045967
AveBedrms            -0.046701
Latitude             -0.144160
dtype: float64


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg.fit(X_train, y_train)
print(f'R2 score (train): {reg.score(X_train, y_train):.4f}')
print(f'R2 score (test): {reg.score(X_test, y_test):.4f}')

R2 score (train): 0.6489
R2 score (test): 0.6395


Discretization

In [15]:
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=10)

In [16]:
longitude_bins = discretizer.fit_transform(X[['Longitude']]).toarray()
longitude_labels = [f'Longitude{i}' for i in range(10)]
longitude_df = pd.DataFrame(longitude_bins, columns=longitude_labels)
pd.concat([housing_df, longitude_df], axis=1)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,Longitude0,Longitude1,Longitude2,Longitude3,Longitude4,Longitude5,Longitude6,Longitude7,Longitude8,Longitude9
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer

columns = ['Longitude', 'Latitude']
preprocessor = ColumnTransformer([
    ('discretizer', KBinsDiscretizer(n_bins=10), columns)
], remainder='passthrough')

In [18]:
from sklearn.pipeline import Pipeline

model = Pipeline([    
    ('pre', preprocessor),
    ('reg', LinearRegression())
])

In [19]:
model.fit(X_train, y_train)
print(f'R2 score (train): {model.score(X_train, y_train):.4f}')
print(f'R2 score (test): {model.score(X_test, y_test):.4f}')

R2 score (train): 0.6701
R2 score (test): 0.6679


In [20]:
# Extend the list of columns to discretize
columns = ['Longitude', 'Latitude', 'AveOccup', 'RoomsPerIndividual'] 
preprocessor = ColumnTransformer([
    ('discretizer', discretizer, columns)
], remainder='passthrough')

# Redefine the model pipeline with the updated preprocessor
model = Pipeline([    
    ('pre', preprocessor),
    ('reg', LinearRegression())
])

model.fit(X_train, y_train)
print(f'R2 score (train): {model.score(X_train, y_train):.4f}')
print(f'R2 score (test): {model.score(X_test, y_test):.4f}')

R2 score (train): 0.7057
R2 score (test): 0.6912


Error Analysis

In [21]:
# Compute the residuals on the test samples
y_test_pred = model.predict(X_test)
residuals = np.abs(y_test - y_test_pred)

# Add the residuals to the DataFrame
housing_df.loc[X_test.index, 'Residual'] = residuals

# Sort the samples in a descending order of the residuals
housing_df.sort_values('Residual', ascending=False).head(10)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal,Residual
6688,0.4999,28.0,7.677419,1.870968,142.0,4.580645,34.15,-118.08,5.00001,4.672149
459,1.1696,52.0,2.436000,0.944000,1349.0,5.396000,37.87,-122.25,5.00001,4.394115
10574,1.9659,6.0,4.795455,1.159091,125.0,2.840909,33.72,-117.70,5.00001,4.144971
12069,4.2386,6.0,7.723077,1.169231,228.0,3.507692,33.83,-117.55,5.00001,3.420554
19542,1.7679,39.0,5.000000,0.888889,22.0,2.444444,37.63,-120.92,4.50000,3.265618
20325,4.5833,21.0,7.278431,1.082353,863.0,3.384314,34.28,-119.04,5.00001,3.236227
17237,3.8456,27.0,5.627171,1.081716,2591.0,2.646578,34.43,-119.66,5.00001,3.128281
17306,2.7275,17.0,5.574286,1.051429,681.0,1.945714,34.38,-119.55,5.00001,2.953196
20349,7.3004,32.0,5.724138,0.758621,63.0,2.172414,34.17,-119.08,1.25000,2.870557
9421,3.7500,38.0,5.770732,0.956098,628.0,3.063415,37.86,-122.53,4.78600,2.867266


In [22]:
max_value_count = (housing_df['MedHouseVal'] == housing_df['MedHouseVal'].max()).sum()
max_value_count

965